In [1]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, push_notebook, output_notebook
from bokeh.models.widgets import Panel
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from bokeh.layouts import row as bokeh_row
from bokeh.layouts import gridplot

output_notebook()

import numpy as np
import pandas as pd

Loading BokehJS ...

In [2]:
p = figure(x_axis_label='x', y_axis_label='y')

x = np.linspace(0, 2*np.pi, 100)
y = np.cos(x)

p.circle(x, y)

# output_file(r'../plots/plot.html')
push_notebook()

show(p, notebook_handle=True)

/home/vayadi/develompent_tools/miniconda3/envs/odsc/lib/python3.6/site-packages/bokeh/io/notebook.py:242: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  warn("Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()")


In [3]:
p.circle(x, y, size=10,
         fill_color='grey', alpha=0.1, line_color=None,
         hover_fill_color='firebrick', hover_alpha=0.5,
         hover_line_color='white')

hover = HoverTool(tooltips=[('x','@x'), ('y','@y')])

p.add_tools(hover)
# Specify the name of the output file and show the result
# output_file(r'../plots/hover_glyph.html')
push_notebook()
show(p)

In [4]:
p1 = figure(x_axis_label='x', y_axis_label='y')
hover1 = HoverTool(tooltips=None, mode='vline')
p1.add_tools(hover1)
p1.circle(x, np.sin(x), size=10,
         fill_color='grey', alpha=0.1, line_color=None,
         hover_fill_color='firebrick', hover_alpha=0.5,
         hover_line_color='white')

p2 = figure(x_axis_label='x', y_axis_label='y')
hover2 = HoverTool(tooltips=None, mode='vline')
p2.add_tools(hover2)
p2.circle(x, np.cos(x), size=10,
         fill_color='grey', alpha=0.1, line_color=None,
         hover_fill_color='firebrick', hover_alpha=0.5,
         hover_line_color='white')


# Create tab1 from plot p1: tab1
tab1 = Panel(child=p1, title='sin')
tab2 = Panel(child=p2, title='cos')

# Create a Tabs layout: layout
layout = Tabs(tabs=[tab1, tab2])

# Specify the name of the output_file and show the result
# output_file('tabs.html')
push_notebook()
show(layout)

In [5]:
import math
import pandas as pd
# model parameters
S0 = 100.0  # initial index level
T = 10.0  # time horizon
r = 0.05  # risk-less short rate
vol = 0.2  # instantaneous volatility

# simulation parameters
np.random.seed(250000)
gbm_dates = pd.DatetimeIndex(start='30-09-2004',
                             end='31-08-2015',
                             freq='B')
M = len(gbm_dates)  # time steps
dt = 1 / 252.  # fixed for simplicity
df = math.exp(-r * dt)  # discount factor

In [6]:
def simulate_gbm():
    # stock price paths
    rand = np.random.standard_normal((M, I))  # random numbers
    S = np.zeros_like(rand)  # stock matrix
    S[0] = S0  # initial values
    for t in range(1, M):  # stock price paths
        S[t] = S[t - 1] * np.exp((r - vol ** 2 / 2) * dt
                        + vol * rand[t] * math.sqrt(dt))

    gbm = pd.DataFrame(S[:, 0], index=gbm_dates, columns=['index'])
    gbm['returns'] = np.log(gbm['index'] / gbm['index'].shift(1))


    # Realized Volatility (eg. as defined for variance swaps)
    gbm['rea_var'] = 252 * np.cumsum(gbm['returns'] ** 2) / np.arange(len(gbm))
    gbm['rea_vol'] = np.sqrt(gbm['rea_var'])
    gbm = gbm.dropna()
    return gbm

In [7]:
I = 1  # paths

df_index = simulate_gbm()

In [8]:
df_index.head()

,index,returns,rea_var,rea_vol
2004-10-01,98.904295,-0.011018,0.030589,0.174898
2004-10-04,98.444867,-0.004656,0.018026,0.134261
2004-10-05,97.696364,-0.007632,0.016911,0.130041
2004-10-06,98.280594,0.005962,0.014922,0.122158
2004-10-07,99.242342,0.009738,0.016718,0.129296


In [9]:
p_gbm = figure(x_axis_type="datetime", title="Index Price")

p_gbm.grid.grid_line_alpha=0.7
p_gbm.xaxis.axis_label = 'Date'
p_gbm.yaxis.axis_label = 'Price'

p_gbm.line((df_index.index), df_index['index'], color='#A6CEE3', legend='INDEX')
p_gbm.legend.location = "top_left"

hover_gbm = HoverTool(tooltips=[('date','@x'), ('price','@y')])
p_gbm.add_tools(hover_gbm)

p_returns = figure(x_axis_type="datetime", title="Indes Returns")

p_returns.grid.grid_line_alpha=0.7
p_returns.xaxis.axis_label = 'Date'
p_returns.yaxis.axis_label = 'Price'

p_returns.line((df_index.index), df_index['returns'], color='#A6CEE3', legend='INDEX returns')
p_returns.legend.location = "top_left"

hover_returns = HoverTool(tooltips=[('date','@x'), ('return','@y')])
p_returns.add_tools(hover_returns)

push_notebook()
#show(bokeh_row(p_gbm, p_returns))

show(gridplot([p_gbm, p_returns], ncols=2, plot_width=450, plot_height=450))